## Transfer Learning For CNN

Get the current working directory to list the image data

In [37]:
import os
path=os.getcwd()
datapath=path+'/data'
data_list=os.listdir(datapath)
print(data_list)

['cats', 'dogs', 'horses', 'Humans']


Initialising important Variables 

In [38]:
img_rows=224
img_cols=224
epochs=2
channel=3
image_data_list=[]
classes_names_list=[]

storing of the data in the list

In [39]:
import cv2

for data in data_list:
    classes_names_list.append(data)
    print('Loaded images from {} folder\n'.format(data))
    img_list=os.listdir(datapath+'/'+data)
    for image in img_list:
        input_img=cv2.imread(datapath+'/'+data+'/'+image)
        input_img_resize=cv2.resize(input_img,(img_rows,img_cols))
        image_data_list.append(input_img_resize)
num_classes=len(classes_names_list)

Loaded images from cats folder

Loaded images from dogs folder

Loaded images from horses folder

Loaded images from Humans folder



### image preprocessing

In [40]:
import numpy as np
img_data=np.array(image_data_list)
img_data=img_data.astype('float32')
img_data /= 255

In [41]:
print(img_data)

[[[[ 0.15686275  0.17254902  0.15294118]
   [ 0.16470589  0.17254902  0.15686275]
   [ 0.17647059  0.17647059  0.16078432]
   ..., 
   [ 0.7019608   0.81176472  0.81960785]
   [ 0.65490198  0.78823531  0.80000001]
   [ 0.63137257  0.7764706   0.78823531]]

  [[ 0.16078432  0.17647059  0.15686275]
   [ 0.16470589  0.17254902  0.15686275]
   [ 0.17647059  0.17647059  0.16078432]
   ..., 
   [ 0.67843139  0.78431374  0.80000001]
   [ 0.64313728  0.77254903  0.78431374]
   [ 0.62352943  0.7647059   0.7764706 ]]

  [[ 0.15686275  0.17254902  0.15294118]
   [ 0.16078432  0.16862746  0.15294118]
   [ 0.16862746  0.16470589  0.14901961]
   ..., 
   [ 0.65490198  0.75686276  0.77254903]
   [ 0.64313728  0.7647059   0.78039217]
   [ 0.64705884  0.7764706   0.79215688]]

  ..., 
  [[ 0.11372549  0.10980392  0.11764706]
   [ 0.10196079  0.09411765  0.10196079]
   [ 0.08627451  0.08235294  0.09019608]
   ..., 
   [ 0.09803922  0.12156863  0.17254902]
   [ 0.08235294  0.10588235  0.15686275]
   [ 0.

In [42]:
img_data=img_data.reshape(img_data.shape[0],img_data.shape[1],img_data.shape[2],channel)
print(img_data.shape)

(808, 224, 224, 3)


In [43]:
num_of_samples=img_data.shape[0]
input_shape=img_data.shape[0]

In [44]:
classes = np.ones((num_of_samples,), dtype='int64')
classes[0:202]=0
classes[202:404]=1
classes[404:606]=2
classes[606:]=3

Convert to one hot encoding

In [45]:
from keras.utils import to_categorical
classes=to_categorical(classes,num_classes)

In [46]:
## shuffling the data
from sklearn.utils import shuffle
x,y = shuffle(img_data,classes,random_state=2)

splitting the data

In [47]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

## Resnet50
### Trasfer Learning - Model 1

In [48]:
from keras.layers import Input
img_input=Input(shape=(224,224,3))

In [49]:
from keras.applications.resnet50 import ResNet50
resnet_model=ResNet50(include_top=True, weights='imagenet', input_tensor=img_input, input_shape=None, pooling='max')

### Aruguments
* include_top: whether to include the fully-connected layer at the top of the network.
* weights: imagenet used for pre-training on ImageNet
* input_tensor: optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.
* input_shape: optional shape tuple, only to be specified if include_top is False

In [50]:
resnet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_50 (Activation)      (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [51]:
from keras.layers.core import Dense
last_layer = resnet_model.get_layer('fc1000').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)

In [52]:
from keras.models import Model
custom_resnet_model=Model(img_input,out)
custom_resnet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_50 (Activation)      (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [53]:
# freeze all the layers except the dense layers
for layer in custom_resnet_model.layers[:-3]:
    layer.trainable = False

In [54]:
custom_resnet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_50 (Activation)      (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [55]:
custom_resnet_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
resnet_fit=custom_resnet_model.fit(x_train,y_train,batch_size=32,epochs=20,verbose=1,validation_data=(x_test,y_test))

Train on 646 samples, validate on 162 samples
Epoch 1/20
646/646 [==============================] - 1106s 2s/step - loss: 1.3407 - acc: 0.8344 - val_loss: 1.3857 - val_acc: 0.2963
Epoch 2/20
646/646 [==============================] - 1175s 2s/step - loss: 1.2875 - acc: 0.9505 - val_loss: 1.3863 - val_acc: 0.2901
Epoch 3/20
646/646 [==============================] - 1232s 2s/step - loss: 1.2432 - acc: 0.9659 - val_loss: 1.3873 - val_acc: 0.1543
Epoch 4/20
288/646 [============>.................] - ETA: 8:59 - loss: 1.2148 - acc: 0.9688

In [ ]:
(loss,accuracy)=custom_resnet_model.evaluate(x_test,y_test,batch_size=10, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))

## VGG19
### Transfer Learning - Model 2

In [ ]:
from keras.applications.vgg19 import VGG19
vgg19_model=VGG19(include_top=True, weights='imagenet', input_tensor=img_input,pooling='max')

In [ ]:
vgg19_model.summary()

In [ ]:
last_layer = vgg19_model.get_layer('block5_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_vgg19 = Model(img_input, out)

In [ ]:
custom_vgg19.summary()

In [ ]:
for layers in custom_vgg19.layers[:-3]:
    layers.trainable=False

In [ ]:
custom_vgg19.summary()

In [ ]:
custom_vgg19.compile(loss='categorical_crossentropy',optimizer='adam',metric=['accuracy'])

In [ ]:
vgg19_fit=custom_vgg19.fit(x_train,y_train,batch_size=32,epochs=32,verbose=1,validation_data=(x_test,y_test))

In [ ]:
(loss, accuracy) = custom_vgg19.evaluate(x_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

## Inceptionv3
### Transfer Learning - Model3

In [ ]:
from keras.applications.inception_v3 import InceptionV3
inception_model=Inceptionv3((include_top=True, weights='imagenet', input_tensor=img_input,pooling='max')

In [ ]:
inception_model.summary()

In [ ]:
last_layer = inception_model.get_layer('').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_inception_model=Model(img_input,out)

In [ ]:
custom_inception_model.summary()

In [ ]:
for layers in custom_inception_model[:-3]:
    layers.trainable=False

In [ ]:
custom_inception_model.summary()

In [ ]:
custom_inception_model.compile(loss='categorical_crossentropy',optimizer='adam',metric=['accuracy'])

In [ ]:
inception_fit=custom_inception_model.fit(x_train,y_train,batch_size=32,epochs=32,verbose=1,validation_data=(x_test,y_test))

In [ ]:
(loss, accuracy) = custom_inception_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

## InceptionResNetV2
### Transfer Learning -model 4

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
inceptionResnt_model=InceptionResNetV2(include_top=True, weights='imagenet', input_tensor=img_input,pooling='max')

In [ ]:
inceptionResnet_model.summary()

In [ ]:
last_layer = inceptionResnet_model.get_layer('').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_inceptionResnt_model=Model(img_input,out)

In [ ]:
custom_inceptionResnet_model.summary()

In [ ]:
for layers in custom_inceptionResnet_model[:-3]:
    layers.trainable=False

In [ ]:
custom_inceptionResnet_model.summary()

In [ ]:
custom_inceptionResnet_model.compile(loss='categorical_crossentropy',optimizer='adam',metric=['accuracy'])

In [ ]:
inceptionResnet_fit=custom_inceptionResnet_model.fit(x_train,y_train,batch_size=32,epochs=32,verbose=1,validation_data=(x_test,y_test))

In [ ]:
(loss, accuracy) = custom_inceptionRenet_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

## Xception
### Transfer Learning -model5

In [ ]:
from keras.applications.xception import Xception
xception_model=Xception(include_top=True, weights='imagenet', input_tensor=img_input,pooling='max')

In [ ]:
xception_model.summary()

In [ ]:
last_layers=xception_model.get_layers('').output
x=Flatten(name='flatten')(last_layers)
x=Dense(128,activation='relu',name='fc1')(x)
x=Dense(128,activation='relu',name='fc2')(x)
out=Dense(num_classes,activation='softmax',name='output')(x)
custom_xception_model=Model(img_input,out)

In [ ]:
custom_xception_model.summary()

In [ ]:
for layers in custom_xception_model[:-3]:
    layers.trainable=False

In [ ]:
custom_xception_model.summary()

In [ ]:
custom_xception_model.compile(loss='categorical_crossentropy',optimizer='adam',metric=['accuracy'])

In [ ]:
xception_fit=custom_xception_model.fit(x_train,y_train,batch_size=32,epochs=32,verbose=1,validation_data=(x_test,y_test))

In [ ]:
(loss,accuracy)=custom_xception_model.evaluate(x_test,y_test,batch_size=10, verbose=1)
print('[INFO] loss={:.4f},acurracy={:.4f}%'.format(loss,accuracy*100))